# IMUs on SMPL

In [ ]:
# Preparing the environment
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

import sys, os
import torch
import numpy as np
import trimesh
from notebook_tools import show_image

from human_body_prior.tools.omni_tools import colors
from human_body_prior.mesh import MeshViewer
from human_body_prior.mesh.sphere import points_to_spheres
from human_body_prior.body_model.body_model import BodyModel
from human_body_prior.tools.omni_tools import copy2cpu as c2c

In [ ]:
comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

bm_path = '../body_models/smplh/male/model.npz'
dmpl_path = '../body_models/dmpls/male/model.npz'

num_betas = 10 # number of body parameters
num_dmpls = 8 # number of DMPL parameters

bm = BodyModel(bm_path=bm_path, num_betas=num_betas, num_dmpls=num_dmpls, path_dmpl=dmpl_path).to(comp_device)
faces = c2c(bm.f)

npz_bdata_path = '/home/lisca/data/amass/CMU/01/01_01_poses.npz' # the path to body data

bdata = np.load(npz_bdata_path)
# print('Data keys available:%s'%list(bdata.keys()))
# print('Vector poses has %d elements for each of %d frames.'%(bdata['poses'].shape[1], bdata['poses'].shape[0]))
# print('Vector dmpls has %d elements for each of %d frames.'%(bdata['dmpls'].shape[1], bdata['dmpls'].shape[0]))
# print('Vector trams has %d elements for each of %d frames.'%(bdata['trans'].shape[1], bdata['trans'].shape[0]))
# print('Vector betas has %d elements constant for the whole sequence.'%bdata['betas'].shape[0])
# print('The subject of the mocap sequence is %s.'%bdata['gender'])

fId = 100 # frame id of the mocap sequence

root_orient = torch.Tensor(bdata['poses'][fId:fId+1, :3]).to(comp_device) # controls the global root orientation
pose_body = torch.Tensor(bdata['poses'][fId:fId+1, 3:66]).to(comp_device) # controls the body
pose_hand = torch.Tensor(bdata['poses'][fId:fId+1, 66:]).to(comp_device) # controls the finger articulation
betas = torch.Tensor(bdata['betas'][:10][np.newaxis]).to(comp_device) # controls the body shape
dmpls = torch.Tensor(bdata['dmpls'][fId:fId+1]).to(comp_device) # controls soft tissue dynamics

imw, imh=1600, 1600
mv = MeshViewer(width=imw, height=imh, use_offscreen=True)

body = bm(pose_body=pose_body, betas=betas)

body_mesh = trimesh.Trimesh(vertices=c2c(body.v[0]), faces=faces, vertex_colors=np.tile(colors['grey'], (6890, 1)))

jnts = c2c(body.Jtr[0])
jnts_msh = points_to_spheres(jnts, vc = colors['red'], radius=0.005)

indcs = torch.tensor([3021, 5431, 1962, 412, 4583, 1096]).to(comp_device)
vrtcs = c2c(body.v[0][indcs])
vrtcs_msh = points_to_spheres(vrtcs, vc = colors['green'], radius=0.01)

mv.set_static_meshes([body_mesh] + jnts_msh + vrtcs_msh)
body_image = mv.render(render_wireframe=True)
show_image(body_image)